    async def get_enhanced_ect(
        self,
        ticker: str,
        year: int,
        quarter: str,
        as_structure: bool = False,
        response_as_html: bool = False
    ) -> Union[List[Any], Dict[str, Any]]:
        """Get enhanced ECT with customer and supplier analysis"""
        
        # 1. Get company relationships using RelationDB
        rel_db = RelationDB(year=year, quarter=quarter, use_cache=True)
        id_bb_company = rel_db.ticker_to_bbg_cid(ticker)
        if not id_bb_company:
            return [f"Could not find company ID for ticker {ticker}"]
        
        company_name = rel_db.d["bbg cid to name map"].get(id_bb_company)
        if not company_name:
            return [f"Could not find company name for ID {id_bb_company}"]

        # 2. Get customer and supplier data separately
        customers = rel_db.get_customers_by_ticker(ticker)  # Returns Dict[str, RelationshipData]
        suppliers = rel_db.get_suppliers_by_ticker(ticker)  # Returns Dict[str, RelationshipData]

        # 3. Get main company ECT
        company_ect = await self._get_company_ect(id_bb_company)
        if isinstance(company_ect, str):  # Error message
            return [company_ect]

        # 4. Process customer ECTs
        customer_ects = await self._get_relationship_ects(
            customers, 
            "customer", 
            rel_db
        )

        # 5. Process supplier ECTs
        supplier_ects = await self._get_relationship_ects(
            suppliers, 
            "supplier", 
            rel_db
        )

        # 6. Generate summary prompts with all data
        summary_responses = await self._generate_summaries(
            company_name=company_name,
            id_bb_company=id_bb_company,
            customers=customers,
            suppliers=suppliers,
            customer_ects=customer_ects,
            supplier_ects=supplier_ects,
            rel_db=rel_db
        )

        # 7. Format output
        if response_as_html:
            company_ect = self._convert_to_html(company_ect)
            for resp in summary_responses:
                resp["result"] = self._convert_to_html(resp["result"])

        if as_structure:
            return [
                company_ect,
                summary_responses[0]["result"] if summary_responses else "",
                summary_responses[1]["result"] if len(summary_responses) > 1 else ""
            ]
        return [company_ect] + [r["result"] for r in summary_responses]

    async def _get_company_ect(self, company_id: str) -> Union[str, Dict]:
        """Get ECT for main company"""
        documents = await self.get_earnings_by_company_id(company_id)
        if not documents:
            return "No earnings documents found"
            
        latest_doc = max(
            documents, 
            key=lambda x: datetime.strptime(x.document_date_s, "%Y-%m-%d")
        )
        return await self.get_ect_summary(latest_doc.refinitiv_id)

    async def _get_relationship_ects(
        self,
        relationships: Dict[str, RelationshipData],
        rel_type: Literal["customer", "supplier"],
        rel_db: RelationDB
    ) -> List[Dict]:
        """Get ECTs for customer/supplier companies"""
        ects = []
        for company_id, rel_data in relationships.items():
            if refinitiv_id := rel_db.ticker_to_refinitive_id(rel_data.company_info.ticker):
                documents = await self.get_earnings_by_company_id(company_id)
                if documents:
                    latest_doc = max(
                        documents, 
                        key=lambda x: datetime.strptime(x.document_date_s, "%Y-%m-%d")
                    )
                    ect = await self.get_ect_summary(refinitiv_id)
                    if not isinstance(ect, str):  # Not an error message
                        ects.append({
                            "company_id": company_id,
                            "name": rel_data.company_info.name,
                            "sector": rel_data.company_info.sector,
                            "ranking": rel_data.company_info.ranking,
                            "consensus": rel_data.consensus_data,
                            "ect": ect
                        })
        return ects

    async def _generate_summaries(
        self,
        company_name: str,
        id_bb_company: str,
        customers: Dict[str, RelationshipData],
        suppliers: Dict[str, RelationshipData],
        customer_ects: List[Dict],
        supplier_ects: List[Dict],
        rel_db: RelationDB
    ) -> List[Dict]:
        """Generate customer and supplier summaries"""
        summary_prompts = []

        if customers:
            customer_prompt = self._generate_customer_summary_prompt(
                company_name=company_name,
                customer_names=", ".join(x["name"] for x in customer_ects),
                customers_cons_data=str([x["consensus"] for x in customer_ects]),
                customer_responses=str([x["ect"] for x in customer_ects]),
                bbg_cid_to_name_map=rel_db.d["bbg cid to name map"],
                id_bb_company=id_bb_company
            )
            summary_prompts.append({
                "type": "Customer",
                "prompt": customer_prompt,
                "company": id_bb_company
            })

        if suppliers:
            supplier_prompt = self._generate_supplier_summary_prompt(
                company_name=company_name,
                supplier_names=", ".join(x["name"] for x in supplier_ects),
                suppliers_cons_data=str([x["consensus"] for x in supplier_ects]),
                supplier_responses=str([x["ect"] for x in supplier_ects]),
                bbg_cid_to_name_map=rel_db.d["bbg cid to name map"],
                id_bb_company=id_bb_company
            )
            summary_prompts.append({
                "type": "Supplier",
                "prompt": supplier_prompt,
                "company": id_bb_company
            })

        if summary_prompts:
            return await self.execute_prompts(
                summary_prompts,
                title="Supply Chain Final Summary"
            )
        return []